In [ ]:
#Importamos librerias necesarias 
from pyspark.sql import SparkSession, functions as F 
import numpy as np

In [ ]:
# Inicializa la sesión de Spark 
spark = SparkSession.builder.appName('Tarea3').getOrCreate() 
spark.conf.set("spark.sql.debug.maxToStringFields", 100)

In [ ]:
# Define la ruta del archivo .csv en HDFS 
file_path = 'hdfs://localhost:9000/Tarea3/rows.csv' 
# Lee el archivo .csv 
df = spark.read.format('csv').option('header','true').option('inferSchema', 'true').load(file_path) 

In [ ]:
#imprimimos el esquema 
df.printSchema() 
# Muestra las primeras filas del DataFrame 
df.show() 

In [ ]:
# Estadisticas básicas 
df.summary().show()

In [ ]:
# Consulta: Filtrar por valor asignado mayor a 50000000 y seleccionar columnas relevantes
print("Subsidios con valor mayor a 50000000\n")
subsidios_filtrados = df.filter(F.col('Valor Asignado') > 50000000).select(
    'Valor Asignado', 'Año de Asignación', 'Departamento', 'Municipio')
subsidios_filtrados.show()

In [ ]:
# Ordenar filas por los valores en la columna "Valor Asignado" en orden descendente
print("Subsidios ordenados de mayor a menor por valor asignado\n")
subsidios_ordenados = df.sort(F.col("Valor Asignado").desc())
subsidios_ordenados.show()

In [ ]:
# Contar cuántos subsidios se asignaron por año
print("Número de subsidios asignados por año\n")
subsidios_por_anio = df.groupBy('Año de Asignación').count().orderBy('Año de Asignación')
subsidios_por_anio.show()

In [ ]:
# Suma total del valor asignado por departamento
print("Suma total de valor asignado por departamento\n")
valor_por_departamento = df.groupBy('Departamento').agg(F.sum('Valor Asignado').alias('Total Asignado')).orderBy(F.col('Total Asignado').desc())
valor_por_departamento.show()

In [ ]:
# Filtrar subsidios asignados a un departamento específico
departamento_filtro = 'Antioquia'  # Puedes cambiar el valor
estado_filtro = 'Asignados' # Puesdes cambiar el valor
print(f"Subsidios asignados en {departamento_filtro}\n")
subsidios_departamento = df.filter((F.col('Departamento') == departamento_filtro)&(F.col('Estado de Postulación') == estado_filtro))
subsidios_departamento.show()

In [ ]:
# Identificar el municipio con mayor cantidad de subsidios
print("Municipio con mayor cantidad de subsidios asignados\n")
municipio_mas_subsidios = df.groupBy('Municipio').count().orderBy(F.col('count').desc()).limit(1)
municipio_mas_subsidios.show()

In [ ]:
# Valor promedio de los subsidios por año
print("Valor promedio de subsidios por año\n")
valor_promedio_anio = df.groupBy('Año de Asignación').agg(F.avg('Valor Asignado').alias('Valor Promedio')).orderBy('Año de Asignación')
valor_promedio_anio.show()

In [ ]:
# Filtrar subsidios por un rango de años (Ejemplo: 2015-2020)
print("Subsidios asignados entre 2015 y 2020\n")
subsidios_rango_anio = df.filter((F.col('Año de Asignación') >= 2015) & (F.col('Año de Asignación') <= 2020))
subsidios_rango_anio.show()

In [ ]:
# Operaciones adicionales de transformación y EDA
# 1. Conversión del DataFrame a RDD para operaciones más complejas
rdd = df.rdd

In [ ]:
# 2. Cálculo de la mediana del valor asignado
valor_asignado_array = np.array(rdd.map(lambda row: float(row['Valor Asignado'])).collect())
mediana_valor = np.median(valor_asignado_array)
print(f"Mediana del valor asignado: {mediana_valor}")

In [ ]:
# 3. Identificación de los 5 departamentos con mayor valor total asignado
top_5_departamentos = df.groupBy('Departamento').agg(F.sum('Valor Asignado').alias('Total Asignado')) \
                         .orderBy(F.col('Total Asignado').desc()) \
                         .limit(5)
print("Top 5 departamentos con mayor valor total asignado:")
top_5_departamentos.show()

In [ ]:
# 4. Análisis de correlación entre el número de hogares y el valor asignado
hogares_vs_valor = df.select('Hogares', 'Valor Asignado')
correlacion = hogares_vs_valor.stat.corr('Hogares', 'Valor Asignado')
print(f"Correlación entre Hogares y Valor Asignado: {correlacion}")

In [ ]:
# 5. Distribución de valores asignados por programa
programas_distribucion = df.groupBy('Programa').agg(F.count('*').alias('Cantidad'), F.sum('Valor Asignado').alias('Total Asignado'))
print("Distribución de valores asignados por programa:")
programas_distribucion.orderBy(F.col('Cantidad').desc()).show()

In [ ]:
# 6. Identificación de subsidios con valores extremadamente altos o bajos
q1 = df.approxQuantile('Valor Asignado', [0.25], 0)[0]
q3 = df.approxQuantile('Valor Asignado', [0.75], 0)[0]
iqr = q3 - q1
outliers_bajos = df.filter((F.col('Valor Asignado') < (q1 - 1.5*iqr)))
outliers_altos = df.filter((F.col('Valor Asignado') > (q3 + 1.5*iqr)))

print("Outliers de valores bajos:")
outliers_bajos.show()
print("Outliers de valores altos:")
outliers_altos.show()

In [ ]:
# 7. Cálculo de la desviación estándar del valor asignado por departamento
std_valor_departamento = df.groupBy('Departamento').agg(F.stddev('Valor Asignado').alias('Desviacion Estandar'))
print("Desviación estándar del valor asignado por departamento:")
std_valor_departamento.orderBy(F.col('Desviacion Estandar').desc()).show()